In [ ]:
import sys
path2cpp_pkg = "/Users/mariusmahiout/Documents/repos/ising_core/build"
sys.path.append(path2cpp_pkg)
import ising

import os
os.chdir("/Users/mariusmahiout/Documents/repos/ising_core/python")
import src.utils as utils
import src.model_eval as eval
import src.isingfitter as fitter
import src.misc_plotting as plotting

os.chdir("..")


import numpy as np
import plotly
from IPython.display import display, HTML

plotly.offline.init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))

In [ ]:
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
import seaborn as sns
sns.set_style(rc = {'axes.facecolor': '#e5ecf6'})
sns.set_theme()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
import pandas as pd
import networkx as nx
import warnings

In [ ]:
def single_run_neq(num_units, beta, num_sims, num_burn, z=3):

    ##############
    # SIMULATION #
    ##############

    # setting up model
    h = np.zeros(num_units)
    z = 3   
    graph = nx.random_regular_graph(d=z, n=num_units)
    J = np.zeros((num_units, num_units))
    for i, j in graph.edges():
        J[i, j] = np.random.uniform(-beta, beta)
        J[j, i] = J[i, j]
    true_model = ising.NeqModel(J, h)

    # simulating
    true_sim = true_model.simulate(num_sims, num_burn)

    lr = 0.1
    win_size = 10
    tol_ml = 1e-3
    max_steps = 3000

    ##############
    # LIKELIHOOD #
    ##############

    # setting up model
    h_init = np.random.uniform(-1.5, 1.5, num_units)
    J_init = np.random.normal(0,  1,  (num_units, num_units))
    J_init = (J_init.T + J_init) * np.sqrt(2) / 2
    np.fill_diagonal(J_init, 0)



    ml_model = ising.NeqModel(J_init, h_init)

    ml_fitter = fitter.NeqFitter(ml_model)
    try:
        with warnings.catch_warnings():
            warnings.filterwarnings('error', category=RuntimeWarning)
            ml_fitter.TAP(true_sim)
    except RuntimeWarning:
        ml_fitter.naive_mean_field(true_sim)


    # # inference
    use_llh = False
    ml_fitter.maximize_likelihood(
        sample=true_sim, 
        max_steps=5000, 
        learning_rate=0.075,
        win_size = win_size,
        tolerance= tol_ml, 
        calc_llh=use_llh
    )


    nmf_model = ising.NeqModel(J_init, h_init)
    nmf_fitter = fitter.NeqFitter(nmf_model)
    nmf_fitter.naive_mean_field(true_sim)

    tap_model = ising.NeqModel(J_init, h_init)
    tap_fitter = fitter.NeqFitter(tap_model)
    tap_fitter.TAP(true_sim)
    
    rmse_nmf = utils.get_rmse(true_model.getCouplings().flatten(), nmf_model.getCouplings().flatten())
    rmse_tap = utils.get_rmse(true_model.getCouplings().flatten(), tap_model.getCouplings().flatten())
    rmse_ml = utils.get_rmse(true_model.getCouplings().flatten(), ml_model.getCouplings().flatten())

    r2_nmf = utils.get_r_squared(true_model.getCouplings().flatten(), nmf_model.getCouplings().flatten())
    r2_tap = utils.get_r_squared(true_model.getCouplings().flatten(), tap_model.getCouplings().flatten())
    r2_ml = utils.get_r_squared(true_model.getCouplings().flatten(), ml_model.getCouplings().flatten())

    rec_err_nmf = utils.get_reconstruction_err(true_model.getCouplings(), nmf_model.getCouplings())
    rec_err_tap = utils.get_reconstruction_err(true_model.getCouplings(), tap_model.getCouplings())
    rec_err_ml = utils.get_reconstruction_err(true_model.getCouplings(), ml_model.getCouplings())


    return {
        "rmse" : {
            "nmf" : rmse_nmf,
            "tap" : rmse_tap,
            "ml" : rmse_ml
        },
        "r2" : {
            "nmf" : r2_nmf,
            "tap" : r2_tap,
            "ml" : r2_ml
        },
        "gamma" : {
            "nmf" : rec_err_nmf,
            "tap" : rec_err_tap,
            "ml" : rec_err_ml
        }

    }


In [ ]:
def single_run_eq(num_units, beta, num_sims, num_burn, z=3):

    ##############
    # SIMULATION #
    ##############

    # setting up model
    h = np.zeros(num_units)
    # J = np.random.uniform(-beta, beta, (num_units, num_units))
    # for i in range(num_units):
    #     J[i, i] = 0
    #     for j in range(i+1, num_units):
    #         J[j, i] = J[i, j]

    graph = nx.random_regular_graph(d=z, n=num_units)
    J = np.zeros((num_units, num_units))
    for i, j in graph.edges():
        J[i, j] = np.random.uniform(-beta, beta)
        J[j, i] = J[i, j]
    true_model = ising.EqModel(J, h)

    # simulating
    true_sim = true_model.simulate(num_sims, num_burn)

    win_size = 10
    tol_ml = 1e-3
    max_steps = 2000

    ##############
    # LIKELIHOOD #
    ##############

    # setting up model
    h_init = np.random.uniform(-1.5, 1.5, num_units)
    J_init = np.random.normal(0,  1,  (num_units, num_units))
    J_init = (J_init.T + J_init) * np.sqrt(2) / 2
    np.fill_diagonal(J_init, 0)

    ml_model = ising.EqModel(J_init, h_init)

    ml_fitter = fitter.EqFitter(ml_model)
    try:
        with warnings.catch_warnings():
            warnings.filterwarnings('error', category=RuntimeWarning)
            ml_fitter.TAP(true_sim)
    except RuntimeWarning:
        ml_fitter.naive_mean_field(true_sim)

    # inference
    ml_fitter.maximize_likelihood(
        sample=true_sim, 
        max_steps=5000, 
        learning_rate=0.075,
        win_size = win_size,
        tolerance= tol_ml, 
        num_sims=30_000, 
        num_burn=5_000,
        calc_llh=False
    )

    ml_fitter.maximize_likelihood(
        sample=true_sim, 
        max_steps=2000, 
        learning_rate=0.05,
        win_size = win_size,
        tolerance= tol_ml, 
        num_sims=50_000, 
        num_burn=5_000,
        calc_llh=False
    )

    ml_fitter.maximize_likelihood(
        sample=true_sim, 
        max_steps=2000, 
        learning_rate=0.025,
        win_size = win_size,
        tolerance= tol_ml, 
        num_sims=50_000, 
        num_burn=5_000,
        calc_llh=False
    )

    nmf_model = ising.EqModel(J_init, h_init)
    nmf_fitter = fitter.EqFitter(nmf_model)
    nmf_fitter.naive_mean_field(true_sim)

    tap_model = ising.EqModel(J_init, h_init)
    tap_fitter = fitter.EqFitter(tap_model)
    tap_fitter.TAP(true_sim)
    
    rmse_nmf = utils.get_rmse(true_model.getCouplings().flatten(), nmf_model.getCouplings().flatten())
    rmse_tap = utils.get_rmse(true_model.getCouplings().flatten(), tap_model.getCouplings().flatten())
    rmse_ml = utils.get_rmse(true_model.getCouplings().flatten(), ml_model.getCouplings().flatten())

    r2_nmf = utils.get_r_squared(true_model.getCouplings().flatten(), nmf_model.getCouplings().flatten())
    r2_tap = utils.get_r_squared(true_model.getCouplings().flatten(), tap_model.getCouplings().flatten())
    r2_ml = utils.get_r_squared(true_model.getCouplings().flatten(), ml_model.getCouplings().flatten())

    rec_err_nmf = utils.get_reconstruction_err(true_model.getCouplings(), nmf_model.getCouplings())
    rec_err_tap = utils.get_reconstruction_err(true_model.getCouplings(), tap_model.getCouplings())
    rec_err_ml = utils.get_reconstruction_err(true_model.getCouplings(), ml_model.getCouplings())

    return {
        "rmse" : {
            "nmf" : rmse_nmf,
            "tap" : rmse_tap,
            "ml" : rmse_ml
        },
        "r2" : {
            "nmf" : r2_nmf,
            "tap" : r2_tap,
            "ml" : r2_ml
        },
        "gamma" : {
            "nmf" : rec_err_nmf,
            "tap" : rec_err_tap,
            "ml" : rec_err_ml
        }

    }

# NEQ testing

### $\beta$ sensitivity

In [ ]:
num_units = 64
num_sims = 100_000
num_burn = 50_000
beta_range = np.linspace(0.05,5, 30)

In [ ]:
outs = []
for beta in tqdm(beta_range):
    out = single_run_neq(num_units, beta, num_sims, num_burn)
    outs.append(out)

In [ ]:
gammas_nmf = [out["gamma"]["nmf"] for out in outs]
gammas_tap = [out["gamma"]["tap"] for out in outs]
gammas_ml = [out["gamma"]["ml"] for out in outs]

In [ ]:
fig, ax = plt.subplots()
ax.plot(beta_range, gammas_nmf, color="blue",marker='o', label="nMF")
ax.plot(beta_range, gammas_tap, color="green",marker='s', label="TAP")
ax.plot(beta_range, gammas_ml, color="red", marker='d', label="ML")

ax.set_ylim([-0.1, 1])
ax.set_xlim([-0.1, 3])
ax.set_xlabel(r"$\beta$", fontsize=20)
ax.set_ylabel(r"$\gamma_J$", fontsize=20)
fig.legend()
fig.tight_layout()
#plt.savefig("neq_sensitivity_beta1.pdf")

### $N$ sensitivity

In [ ]:
#num_units = 64
num_sims = 100_000
num_burn = 50_000
beta = 1.3
n_range = range(4, 200 + 1, 2)

In [ ]:
outs = []
for num_units in tqdm(n_range):
    out = single_run_neq(num_units, beta, num_sims, num_burn)

    outs.append(out)

In [ ]:
gammas_nmf = [out["gamma"]["nmf"] for out in outs]
gammas_tap = [out["gamma"]["tap"] for out in outs]
gammas_ml = [out["gamma"]["ml"] for out in outs]

fig, ax = plt.subplots()
ax.plot(n_range, gammas_nmf, color="blue", marker='o', label="nMF")
ax.plot(n_range, gammas_tap, color="green", marker='s', label="TAP")
ax.plot(n_range, gammas_ml, color="red", marker='d', label="ML")

ax.set_ylim([-0.05, 0.65])
ax.set_xlim([2, 200])
ax.set_xlabel(r"$N$", fontsize=20)
ax.set_ylabel(r"$\gamma_J$", fontsize=20)
fig.legend()
fig.tight_layout()
plt.savefig("neq_sensitivity_size.pdf")

# EQ testing

In [ ]:
outs = []
for beta in tqdm(beta_range):
    out = single_run_eq(num_units, beta, num_sims, num_burn)
    outs.append(out)

In [ ]:
rmses_nmf = [out["gamma"]["nmf"] for out in outs]
rmses_tap = [out["gamma"]["tap"] for out in outs]
rmses_ml = [out["gamma"]["ml"] for out in outs]

In [ ]:
plt.plot(beta_range, rmses_nmf, color="dodgerblue", label="nMF")
plt.plot(beta_range, rmses_tap, color="green", label="TAP")
plt.plot(beta_range, rmses_ml, color="red", label="ML")


plt.ylim([0.4, 1])
plt.xlim([0.1, 2])
plt.xlabel(r"$\beta$", fontsize=20)
plt.ylabel(r"$\gamma_J$", fontsize=20)
plt.legend()
#plt.savefig("beta_sens.pdf")

In [ ]:





fig, ax = plt.subplots()
ax.plot(beta_range, rmses_nmf, color="blue",marker='o', label="nMF")
ax.plot(beta_range, rmses_tap, color="green",marker='s', label="TAP")
ax.plot(beta_range, rmses_ml, color="darkorange", marker='d', label="ML")

ax.set_ylim([0.4, 1])
ax.set_xlim([0.1, 2])
ax.set_xlabel(r"$\beta$", fontsize=20)
ax.set_ylabel(r"$\gamma_J$", fontsize=20)
fig.legend()
fig.tight_layout()
plt.savefig("eq_sensitivity_beta.pdf")

In [ ]:
r2s_nmf = [out["r2"]["nmf"] for out in outs]
r2s_tap = [out["r2"]["tap"] for out in outs]
r2s_ml = [out["r2"]["ml"] for out in outs]

rmses_nmf = [out["rmse"]["nmf"] for out in outs]
rmses_tap = [out["rmse"]["tap"] for out in outs]
rmses_ml = [out["rmse"]["ml"] for out in outs]

fig, ax = plt.subplots(ncols=2, figsize=(10,5))
ax[0].plot(beta_range, rmses_nmf, color="blue",marker='o', label="nMF")
ax[0].plot(beta_range, rmses_nmf, color="green",marker='s', label="TAP")
ax[0].plot(beta_range, rmses_nmf, color="darkorange", marker='d', label="ML")

ax[1].plot(beta_range, r2s_nmf, color="blue",marker='o')#, label="nMF")
ax[1].plot(beta_range, r2s_tap, color="green",marker='s')#, label="TAP")
ax[1].plot(beta_range, r2s_ml, color="darkorange", marker='d')#, label="ML")

ax[0].set_ylim([0.0, 1])
ax[0].set_xlim([0.0, 2])

ax[1].set_ylim([0.0, 1])
ax[1].set_xlim([0.1, 2])

ax[0].set_xlabel(r"$\beta$", fontsize=20)
ax[1].set_xlabel(r"$\beta$", fontsize=20)

ax[0].set_ylabel(r"$\gamma_J$", fontsize=20)
fig.legend()
fig.tight_layout()
plt.savefig("eq_sensitivity_beta_yr.pdf")

In [ ]:
# num_units = 64
# num_sims = 50_000
# num_burn = 10_000
# beta_range = np.linspace(0.0,2.5, 30)

# outs = []
# current_betas = []
# for idx, beta in enumerate(tqdm(beta_range)):
#     current_betas.append(beta)
#     out = single_run_eq(num_units, beta, num_sims, num_burn)
#     outs.append(out)

#     rmses_nmf = [out["gamma"]["nmf"] for out in outs]
#     rmses_tap = [out["gamma"]["tap"] for out in outs]
#     rmses_ml = [out["gamma"]["ml"] for out in outs]

#     plt.clf()
#     plt.plot(current_betas, rmses_nmf, alpha=1, color="blue", label="nMF" if idx == len(beta_range) else None)
#     plt.plot(current_betas, rmses_tap, alpha=1, color="green", label="TAP" if idx == len(beta_range) else None)
#     plt.plot(current_betas, rmses_ml, alpha=1, color="red", label="ML" if idx == len(beta_range) else None)

#     plt.ylim([0, 1])
#     plt.xlim([0, 2.5])
#     plt.xlabel(r"$\beta$", fontsize=20)
#     plt.ylabel(r"$\gamma_J$", fontsize=20)
#     plt.savefig("beta_sens_cont.pdf")
#     if beta != beta_range[-1]:
#         plt.clf()

# plt.legend()
# plt.savefig("beta_sens_cont.pdf")

